In [ ]:
import sys
from pathlib import Path
from idh2022fns import compute_nMx, calc_e0, compute_idh_saude
import pandas as pd

In [ ]:
root_dir = Path(__file__).resolve().parents[1]
sys.path.append(str(root_dir))

# ======================================
# 1. Calcular taxas etárias (nMx)
# ======================================
mx = compute_nMx(df_pop, df_obitos)
print("nMx calculadas:", mx.shape)

# ======================================
# 2. Calcular e0 municipal (por sexo)
# ======================================
e0_list = []
for (mun, sexo), grp in mx.groupby(["cod_municipio","sexo"]):
    e0 = calc_e0(grp)
    e0_list.append((mun, sexo, e0))

df_e0 = pd.DataFrame(e0_list, columns=["cod_municipio","sexo","e0"])
print("e0 estimadas:", df_e0.shape)

# ======================================
# 3. Calcular média ponderada por sexo (total municipal)
pop_weights = df_pop.groupby(["cod_municipio","sexo"], as_index=False)["pop"].sum()
df_e0_w = df_e0.merge(pop_weights, on=["cod_municipio","sexo"], how="left")

df_e0_total = (
    df_e0_w.groupby("cod_municipio", as_index=False)
    .apply(lambda g: pd.Series({
        "e0_total": (g["e0"] * g["pop"]).sum() / g["pop"].sum()
    }))
)

# ======================================
# 4. Calcular o IDH-Saúde
idh_saude = compute_idh_saude(df_e0_total)
idh_saude.to_csv(root_dir / "data/idh_saude_2022.csv", index=False)

print(idh_saude.head())
